In [1]:
import os
os.chdir('../..')

In [2]:
from argparse import ArgumentError
from glob import glob
from pathlib import Path
from typing import Counter

import spacy
import srsly
import typer
from spacy.tokens import Doc
from tqdm import tqdm

from scripts.components import (
    component_lowercase_lemmas,
    financial_tone_classifier,
    sentence_masker,
    predict_domain_from_keywords,
    sustainability_potential_classifier,
    keyword_ruler,
    entity_ruler,
)
from scripts.data_helpers import (
    extract_sentence_entities,
    filter_dups,
    filter_item_fields,
    hash_documents,
)
from scripts.extract_paragraphs import paragraph_generator



In [3]:
nlp = spacy.load("en_core_web_trf")
nlp.add_pipe("labeling_ruler")
nlp.add_pipe("labeling_extend_ruler")
nlp.add_pipe("expand_entities")
nlp.add_pipe("lower_case_lemmas", after="lemmatizer")
nlp.add_pipe("keyword_ruler", after="lower_case_lemmas")
nlp.add_pipe("domain_classifier")
nlp.add_pipe("sentence_masker_domain")

/home/oguz/.cache/pypoetry/virtualenvs/smart-evidence-hIlc30vc-py3.9/lib/python3.9/site-packages/spacy/util.py:833: UserWarning: [W095] Model 'en_core_web_trf' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/oguz/.cache/pypoetry/virtualenvs/smart-evidence-hIlc30vc-py3.9/lib/python3.9/site-packages/spacy_transformers/pipeline_component.py:406: UserWarning: Automatically converting a transformer component from spacy-transformers v1.0 to v1.1+. If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spacy-transformers version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
doc = nlp('Cement causes a lot of GHG emissions.')

/home/oguz/.cache/pypoetry/virtualenvs/smart-evidence-hIlc30vc-py3.9/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [5]:
[(ent.label_, ent.ent_id_) for ent in doc.ents]

[('IX_PRODUCT',
  'Products & Activities/UNSPSC/30000000_Structures and Building and Construction and Manufacturing Components and Supplies/30110000_Concrete and cement and plaster/PRODUCT/Cement'),
 ('IX_IMPACT', 'Impact/IRIS/Climate/IMPACT/ghg'),
 ('IX_IMPACT', 'Impact/IRIS/Climate/IMPACT/emission')]